In [67]:
import requests
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import time
import datetime as dt

print('Imports completed!')

Imports completed!


In [68]:
# Read in game score data
game_df = pd.read_excel('./data/apiGameScores.xlsx')
game_df

,game_ids,date,visitor,home,visitor_points,home_points
0,319,2015-11-25T01:00:00.000Z,Boston Celtics,Atlanta Hawks,97.0,121.0
1,1300,2016-04-09T23:30:00.000Z,Boston Celtics,Atlanta Hawks,107.0,118.0
2,1344,2016-04-16T23:00:00.000Z,Boston Celtics,Atlanta Hawks,101.0,102.0
3,1353,2016-04-19T23:00:00.000Z,Boston Celtics,Atlanta Hawks,72.0,89.0
4,1376,2016-04-27T00:30:00.000Z,Boston Celtics,Atlanta Hawks,83.0,110.0
...,...,...,...,...,...,...
14237,9048,2021-04-13T01:00:00.000Z,Washington Wizards,Utah Jazz,125.0,121.0
14238,10012,2021-12-19T02:00:00.000Z,Washington Wizards,Utah Jazz,109.0,103.0
14239,11527,2022-12-23T02:00:00.000Z,Washington Wizards,Utah Jazz,112.0,120.0
14240,13430,2024-03-05T02:00:00.000Z,Washington Wizards,Utah Jazz,115.0,127.0


In [69]:
game_df.isnull().sum()

game_ids             0
date                 0
visitor              0
home                 0
visitor_points    1329
home_points       1329
dtype: int64

In [70]:
# Drop rows with null values
game_df = game_df.dropna()

# Shape of the data frame
game_df.shape

(12913, 6)

In [71]:
# Grab game ids to be used with api call
game_ids = game_df['game_ids'].values
game_ids

array([  319,  1300,  1344, ..., 10012, 11527, 13430], dtype=int64)

In [72]:
url = "https://api-nba-v1.p.rapidapi.com/games/statistics"
headers = {
	"x-rapidapi-key": "dda6cf782dmshebe3119a485b548p154863jsn7c9c31f1e7a7",
	"x-rapidapi-host": "api-nba-v1.p.rapidapi.com"
}

statistics = []
for game_id in game_ids:
	querystring = {"id": game_id}
	response = requests.get(url, headers=headers, params=querystring)
	
 	# Check if the API request was successful
	if response.status_code == 200:
		game_statistics = response.json()
		
		if len(game_statistics.get('response', [])) > 1:
			# Extract game id
			id = game_statistics['parameters']['id']

			# Extract visitor and home team names
			visitor = game_statistics['response'][0].get('team', {}).get('name', 'Unknown')
			home = game_statistics['response'][1].get('team', {}).get('name', 'Unknown')

			# Extract statistics for visitor team
			visitor_statics = game_statistics['response'][0]['statistics'][0]
			visitor_stats = {f"visitor_{key}":value for key, value in visitor_statics.items()}
			visitor_stats['game_id'] = id
			visitor_stats['visitor_team'] = visitor
			
			# Extract statistics for home team
			home_statics = game_statistics['response'][1]['statistics'][0]
			home_stats = {f"home_{key}":value for key, value in home_statics.items()}
			home_stats['game_id'] = id
			home_stats['home_team'] = home

			# Append both visitor and home stats
			statistics.append({**visitor_stats, **home_stats})

			# Sleep function
			time.sleep(5)
   
# Create the Data Frame
game_statistics_df = pd.DataFrame(statistics)

In [73]:
# Export as csv file
game_statistics_df.to_csv('./data/gameStatistics.csv', index=False)
game_statistics_df

,visitor_fastBreakPoints,visitor_pointsInPaint,visitor_biggestLead,visitor_secondChancePoints,visitor_pointsOffTurnovers,visitor_longestRun,visitor_points,visitor_fgm,visitor_fga,visitor_fgp,...,home_defReb,home_totReb,home_assists,home_pFouls,home_steals,home_turnovers,home_blocks,home_plusMinus,home_min,home_team
0,18.0,34.0,2.0,NaN,20.0,NaN,97.0,34.0,82.0,41.5,...,37.0,43.0,33.0,25.0,8.0,20.0,7.0,24,240:00,Atlanta Hawks
1,25.0,46.0,12.0,NaN,18.0,NaN,107.0,40.0,90.0,44.4,...,39.0,44.0,31.0,22.0,10.0,17.0,10.0,11,240:00,Atlanta Hawks
2,15.0,36.0,3.0,NaN,10.0,NaN,101.0,37.0,102.0,36.3,...,40.0,53.0,23.0,20.0,4.0,11.0,9.0,1,240:00,Atlanta Hawks
3,8.0,36.0,0.0,NaN,6.0,NaN,72.0,28.0,88.0,31.8,...,46.0,49.0,20.0,16.0,7.0,15.0,15.0,17,240:00,Atlanta Hawks
4,8.0,32.0,10.0,NaN,9.0,NaN,83.0,29.0,77.0,37.7,...,38.0,51.0,30.0,17.0,13.0,12.0,3.0,27,240:00,Atlanta Hawks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3053,NaN,NaN,NaN,NaN,NaN,NaN,113.0,35.0,76.0,46.1,...,26.0,45.0,20.0,32.0,12.0,23.0,3.0,-18,240:00,Toronto Raptors
3054,6.0,68.0,11.0,18.0,14.0,15.0,127.0,48.0,90.0,53.3,...,28.0,37.0,28.0,18.0,5.0,9.0,4.0,-11,240:00,Washington Wizards
3055,NaN,NaN,NaN,NaN,NaN,NaN,121.0,44.0,88.0,50.0,...,35.0,42.0,33.0,17.0,4.0,12.0,3.0,-9,240:00,Utah Jazz
3056,20.0,52.0,6.0,13.0,20.0,13.0,124.0,48.0,98.0,49.0,...,37.0,45.0,26.0,22.0,6.0,18.0,7.0,4,240:00,Utah Jazz


In [82]:
game_stats = pd.read_csv('./data/gameStatistics.csv')
game_stats

,visitor_fastBreakPoints,visitor_pointsInPaint,visitor_biggestLead,visitor_secondChancePoints,visitor_pointsOffTurnovers,visitor_longestRun,visitor_points,visitor_fgm,visitor_fga,visitor_fgp,...,home_defReb,home_totReb,home_assists,home_pFouls,home_steals,home_turnovers,home_blocks,home_plusMinus,home_min,home_team
0,18.0,34.0,2.0,NaN,20.0,NaN,97.0,34.0,82.0,41.5,...,37.0,43.0,33.0,25.0,8.0,20.0,7.0,24.0,240:00,Atlanta Hawks
1,25.0,46.0,12.0,NaN,18.0,NaN,107.0,40.0,90.0,44.4,...,39.0,44.0,31.0,22.0,10.0,17.0,10.0,11.0,240:00,Atlanta Hawks
2,15.0,36.0,3.0,NaN,10.0,NaN,101.0,37.0,102.0,36.3,...,40.0,53.0,23.0,20.0,4.0,11.0,9.0,1.0,240:00,Atlanta Hawks
3,8.0,36.0,0.0,NaN,6.0,NaN,72.0,28.0,88.0,31.8,...,46.0,49.0,20.0,16.0,7.0,15.0,15.0,17.0,240:00,Atlanta Hawks
4,8.0,32.0,10.0,NaN,9.0,NaN,83.0,29.0,77.0,37.7,...,38.0,51.0,30.0,17.0,13.0,12.0,3.0,27.0,240:00,Atlanta Hawks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3053,NaN,NaN,NaN,NaN,NaN,NaN,113.0,35.0,76.0,46.1,...,26.0,45.0,20.0,32.0,12.0,23.0,3.0,-18.0,240:00,Toronto Raptors
3054,6.0,68.0,11.0,18.0,14.0,15.0,127.0,48.0,90.0,53.3,...,28.0,37.0,28.0,18.0,5.0,9.0,4.0,-11.0,240:00,Washington Wizards
3055,NaN,NaN,NaN,NaN,NaN,NaN,121.0,44.0,88.0,50.0,...,35.0,42.0,33.0,17.0,4.0,12.0,3.0,-9.0,240:00,Utah Jazz
3056,20.0,52.0,6.0,13.0,20.0,13.0,124.0,48.0,98.0,49.0,...,37.0,45.0,26.0,22.0,6.0,18.0,7.0,4.0,240:00,Utah Jazz


In [83]:
game_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3058 entries, 0 to 3057
Data columns (total 55 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   visitor_fastBreakPoints     2319 non-null   float64
 1   visitor_pointsInPaint       2319 non-null   float64
 2   visitor_biggestLead         2319 non-null   float64
 3   visitor_secondChancePoints  2009 non-null   float64
 4   visitor_pointsOffTurnovers  2319 non-null   float64
 5   visitor_longestRun          2009 non-null   float64
 6   visitor_points              3057 non-null   float64
 7   visitor_fgm                 3057 non-null   float64
 8   visitor_fga                 3057 non-null   float64
 9   visitor_fgp                 3057 non-null   float64
 10  visitor_ftm                 3057 non-null   float64
 11  visitor_fta                 3057 non-null   float64
 12  visitor_ftp                 3057 non-null   float64
 13  visitor_tpm                 3057 

In [ ]:
# Looks at the total percentage of null values in each columns
null_percentage = (game_stats.isnull().sum() / len(game_stats)) * 100

# Rule of thumb: Moderate Null Percentage (10-30%)
# Filling the null values with an appropriate value (like the mean, median, or mode) is often better in this range, especially if the column is critical for analysis
# Get columns from null_percentage
null_columns = null_percentage[null_percentage > 20].index

# Fill rows with the average of the column
for column in null_columns:
    average_fill_value = game_stats[column].mean()
    game_stats[column] = game_stats[column].fillna(average_fill_value)

Index(['visitor_pointsInPaint', 'visitor_biggestLead',
       'visitor_secondChancePoints', 'visitor_pointsOffTurnovers',
       'visitor_longestRun', 'home_fastBreakPoints', 'home_pointsInPaint',
       'home_biggestLead', 'home_secondChancePoints',
       'home_pointsOffTurnovers', 'home_longestRun'],
      dtype='object')

In [92]:
# Fill rows with the average of the column
visitor_average_fastBreakPoints = game_stats['visitor_fastBreakPoints'].mean()
game_stats['visitor_fastBreakPoints'] = game_stats['visitor_fastBreakPoints'].fillna(visitor_average_fastBreakPoints)

In [111]:
# Based on the column data and the fact that the home_team has the same missing data as the visitor_team
# A function should be created to fill all the Null values with the average of those columns
# Later will see if this affects how well the machine learning model works

def fillNullCols(df):
    # Find all data type 'int' or 'float' columns that have
    df.select_dtypes(include=['int', 'float'])
    
    # Looks at the total percentage of null values in each columns
    null_percentage = (df.isnull().sum() / len(df)) * 100

    # Rule of thumb: Moderate Null Percentage (10-30%)
    # Filling the null values with an appropriate value (like the mean, median, or mode) is often better in this range, especially if the column is critical for analysis
    # Get columns from null_percentage
    null_columns = null_percentage[null_percentage > 20].index

    # Fill rows with the average of the column
    for column in null_columns:
        average_fill_value = df[column].mean()
        game_stats[column] = df[column].fillna(average_fill_value)
        
    return df


clean_game_stats = fillNullCols(game_stats)

In [112]:
clean_game_stats

,visitor_fastBreakPoints,visitor_pointsInPaint,visitor_biggestLead,visitor_secondChancePoints,visitor_pointsOffTurnovers,visitor_longestRun,visitor_points,visitor_fgm,visitor_fga,visitor_fgp,...,home_defReb,home_totReb,home_assists,home_pFouls,home_steals,home_turnovers,home_blocks,home_plusMinus,home_min,home_team
0,18.00000,34.000000,2.000000,12.816327,20.00000,10.516177,97.0,34.0,82.0,41.5,...,37.0,43.0,33.0,25.0,8.0,20.0,7.0,24.0,240:00,Atlanta Hawks
1,25.00000,46.000000,12.000000,12.816327,18.00000,10.516177,107.0,40.0,90.0,44.4,...,39.0,44.0,31.0,22.0,10.0,17.0,10.0,11.0,240:00,Atlanta Hawks
2,15.00000,36.000000,3.000000,12.816327,10.00000,10.516177,101.0,37.0,102.0,36.3,...,40.0,53.0,23.0,20.0,4.0,11.0,9.0,1.0,240:00,Atlanta Hawks
3,8.00000,36.000000,0.000000,12.816327,6.00000,10.516177,72.0,28.0,88.0,31.8,...,46.0,49.0,20.0,16.0,7.0,15.0,15.0,17.0,240:00,Atlanta Hawks
4,8.00000,32.000000,10.000000,12.816327,9.00000,10.516177,83.0,29.0,77.0,37.7,...,38.0,51.0,30.0,17.0,13.0,12.0,3.0,27.0,240:00,Atlanta Hawks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3053,12.34972,44.878827,11.448038,12.816327,16.35705,10.516177,113.0,35.0,76.0,46.1,...,26.0,45.0,20.0,32.0,12.0,23.0,3.0,-18.0,240:00,Toronto Raptors
3054,6.00000,68.000000,11.000000,18.000000,14.00000,15.000000,127.0,48.0,90.0,53.3,...,28.0,37.0,28.0,18.0,5.0,9.0,4.0,-11.0,240:00,Washington Wizards
3055,12.34972,44.878827,11.448038,12.816327,16.35705,10.516177,121.0,44.0,88.0,50.0,...,35.0,42.0,33.0,17.0,4.0,12.0,3.0,-9.0,240:00,Utah Jazz
3056,20.00000,52.000000,6.000000,13.000000,20.00000,13.000000,124.0,48.0,98.0,49.0,...,37.0,45.0,26.0,22.0,6.0,18.0,7.0,4.0,240:00,Utah Jazz


In [113]:
game_stats.isnull().sum()


visitor_fastBreakPoints       0
visitor_pointsInPaint         0
visitor_biggestLead           0
visitor_secondChancePoints    0
visitor_pointsOffTurnovers    0
visitor_longestRun            0
visitor_points                1
visitor_fgm                   1
visitor_fga                   1
visitor_fgp                   1
visitor_ftm                   1
visitor_fta                   1
visitor_ftp                   1
visitor_tpm                   1
visitor_tpa                   1
visitor_tpp                   1
visitor_offReb                1
visitor_defReb                1
visitor_totReb                1
visitor_assists               1
visitor_pFouls                1
visitor_steals                1
visitor_turnovers             1
visitor_blocks                1
visitor_plusMinus             1
visitor_min                   1
game_id                       0
visitor_team                  0
home_fastBreakPoints          0
home_pointsInPaint            0
home_biggestLead              0
home_sec

In [ ]:
# B